# Zadania

In [1]:
# Two-class MNIST 

from keras.datasets import mnist
import numpy as np

d1 = 5
d2 = 6

(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()

X_train = (mnist_x_train.astype('float32') / 255.).reshape((len(mnist_x_train), np.prod(mnist_x_train.shape[1:])))
y_train = mnist_y_train
X_test = (mnist_x_test.astype('float32') / 255.).reshape((len(mnist_x_test), np.prod(mnist_x_test.shape[1:])))
y_test = mnist_y_test

X_train = X_train[np.logical_or(y_train == d1, y_train == d2)]
y_train = y_train[np.logical_or(y_train == d1, y_train == d2)]
y_train[y_train==d1] = 0
y_train[y_train==d2] = 1
X_test = X_test[np.logical_or(y_test == d1, y_test == d2)]
y_test = y_test[np.logical_or(y_test == d1, y_test == d2)]
y_test[y_test==d1] = 0
y_test[y_test==d2] = 1

print X_train.shape
print X_test.shape


Using TensorFlow backend.


(11339, 784)
(1850, 784)


## Ćwiczenie 1 [5 pkt] ok?

Uzupełnij metody forward_pass oraz backward_pass w klasach ReLU, Sigmoid i Dense. Metoda forward_pass ma przyjmować batch inputów i zwracać batch outputów. Metoda backward_pass ma przyjmować batch inputów oraz batch pochodnych cząstkowych outputów i zwracać batch pochodnych cząstkowych inputów oraz wektor (**nie batch**) pochodnych cząstkowych wag. Jeśli wagi przechowujemy w macierzy dwuwymiarowej, to możemy najpierw policzyć pochodne cząstkowe w macierzy o takim samym kształcie, a następnie np. użyć .flat. 

Uwaga: dla warstw bez wag należy zwrócić None.

## Ćwiczenie 2 [4 pkt] ok ?

Uzupełnij metodę _forward_pass klasy Network. Metoda ta ma przyjmować batch inputów (X) i zwracać dwie rzeczy:
* inps - lista batchów inputów dla każdej warstwy w sieci (włącznie z X); te wartości będziemy używali w metodzie _backward_pass
* output - batch outputów z sieci (czyli $\mathbf{\hat y}$); output **nie** powinien być ostatnim elementem inps.

## Ćwiczenie 3 [5 pkt] ok ?

Uzupełnij metodę _backward_pass klasy Network. Zwróć uwagę, że pochodna funkcji kosztu po neuronach ostatniej warstwy jest już liczona w metodzie _fit_on_batch. Metoda ma zwracać listę layer_grads, której elementy to wektory pochodnych cząstkowych funkcji kosztu po kolejnych warstwach (zwrócone przez metodę Layer.backward_pass). Kolejność wektorów w tej liście ma być zgodna z kolejnością warstw w sieci.

## Ćwiczenie 4 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą i aktywacją Sigmoid na powyższych danych (dwuklasowy MNIST). Użyj MSE jako funkcji kosztu (oznacza to regresję do numeru klasy, co jest złym pomysłem, ale póki nie mamy klasy Crossentropy musi nam to wystarczyć). Użyj GD. Reportuj loss oraz accuracy.

## Ćwiczenie 5 [3 pkt]
Uzupełnić klasę Crossentropy, wzorując się na klasie MSE.

## Ćwiczenie  6 [3 pkt]
Uzupełnić klasę Momentum, wzorując się na klasie GD. Wzory można znaleźć tutaj: http://distill.pub/2017/momentum/

## Ćwiczenie 7 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą. Rozważ dwa przypadki: aktywację ReLU oraz Sigmoid. Czy jest sens używać ReLU jako ostatnią warstwę? Użyj Crossentropy jako funkcji kosztu. Użyj Momentum. Reportuj loss oraz accuracy.

## Ćwiczenie 8 [6 pkt]
Vanishing gradient.

Zadanie polega na zbadaniu zjawiska *vanishing gradient* w głębokich sieciach. Należy zmodyfikować kod warstwy Dense i dodać monitorowanie **normy euklidesowej** wektora delta_weights. Każdą warstwę Dense w trenowanej sieci należy monitorować oddzielnie. Po każdym wywołaniu metody fit_on_batch każdy z monitorów powinien zapamiętać nową normę. Po nauczeniu sieci dla każdej warstwy należy narysować wykres: poziomo - numer wywołania fit_on_batch, pionowo - norma delta_weights. Im niżej znajduje się warstwa Dense, tym silniej będzie zachodziło zjawisko *vanishing gradient*.

Naucz dwuwarstwową sieć z aktywacjami Sigmoid, reportując normy delta_weights. Powtórz to dla głębszej sieci (np. 6-10 warstw).

## Ćwiczenie 9 [4 pkt]
Przetestować kod z ćwiczenia 7. (dwuwarstwowa sieć) stosując inne inicjalizacje wag w warstwach Dense. Napisać własną inicjalizację wag, która sprawi, że sieć niczego się nie nauczy (init='stupid').


In [57]:
# Warstwy

class Layer():

    def forward_pass(self, input):
        # return output
        raise NotImplementedError()

    def backward_pass(self, input, output_grad):
        # return input_grad, weight_grad
        raise NotImplementedError()

    def update_weights(self, delta_weights):
        pass

    def debug_grad(self, evaluate_loss):
        return None

class ReLU(Layer):
    
    def forward_pass(self, input_):
        # Ćwiczenie 1
        from functools import partial
        f = np.vectorize(partial(max, 0))
        return f(input_)
        
    def backward_pass(self, input_, output_grad):
        # Ćwiczenie 1
        fbool = np.vectorize(lambda x: x>0)
        ones = fbool(input_)
        return np.multiply(ones, output_grad), None
    
class Sigmoid(Layer):
    sigma = np.vectorize(lambda x: 1/float(1+np.exp(x)))
    def forward_pass(self, input):
        # Ćwiczenie 1
        return self.sigma(input)

    def backward_pass(self, input_, output_grad):
        # Ćwiczenie 1
        sigm_input = self.sigma(input_)
        return reduce(np.multiply,
            output_grad,
            sigm_input,
            np.subtract(ones(sigm_input.shape), sigm_input)), None

class Dense(Layer):

    def __init__(self, input_size, output_size, init = 'gaussian'):
        input_size += 1
        if init == 'zeros':
            self.weights = np.zeros((input_size, output_size))
        elif init == 'gaussian':
            self.weights = np.random.normal(
                0.,
                2. / (input_size + output_size),
                (input_size, output_size)
            )
        elif init == 'stupid':
            # Ćwiczenie 8
            raise NotImplementedError()            
        else:
            raise NotImplementedError()
        self.weights = np.asmatrix(self.weights)

    def forward_pass(self, input):
        # Ćwiczenie 1
        x = np.hstack(ones(input.shape[0], 1), input)
        return x.T*self.weights
    
    
    def backward_pass(self, input_, output_grad):
        # Ćwiczenie 1
        dLdW = []
        for i in range(output_grad.shape[0]):  #sprawdzic czy to jest output_size
            dLdW.append(np.dot(input_[:, i], output_grad[:, i]))
        dLdW = np.array(dLdW)
#         print output_grad.shape, self.weights.shape
        dLdX = self.weights * output_grad.T
#         input_grad = dLdX[:,1:]
        input_grad = dLdX.T[:,1:]
        weight_grad = dLdW
        return input_grad, weight_grad.flat

    def update_weights(self, delta_weights):
        # Ćwiczenie 7 - monitorowanie normy wektora delta_weights
        print delta_weights.shape, self.weights.shape
        self.weights += delta_weights.reshape(self.weights.shape)

    def debug_grad(self, evaluate_loss):
        base = evaluate_loss()
        grad = []
        for (x, y), w in np.ndenumerate(self.weights):
            self.weights[x, y] = w + 0.0001
            changed = evaluate_loss()
            grad.append(10000. * (changed - base))
            self.weights[x, y] = w
        return grad

In [3]:
# Optimizers

class Optimizer():

    def calculate_deltas(self, grad):
        raise NotImplementedError()

class GD(Optimizer):

    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def calculate_deltas(self, grad):
        return -self.learning_rate * grad

class Momentum(Optimizer):

    def __init__(self, alpha, beta):
        # Ćwiczenie 6
        pass
        
    def calculate_deltas(self, grad):
        # Ćwiczenie 6
        pass
        

In [4]:
# Funkcje kosztu

class Loss():

    def forward_pass(self, y, t):
        # return cost
        raise NotImplementedError()

    def backward_pass(self, y, t):
        # return y_grad
        raise NotImplementedError()

class MSE(Loss):

    def forward_pass(self, y, t):
        return np.average(0.5 * np.square(y - t))

    def backward_pass(self, y, t):
        return (y - t) / y.size

class Crossentropy(Loss):

    def forward_pass(self, y, t):
        # Ćwiczenie 5
        pass
        
    def backward_pass(self, y, t):
        # Ćwiczenie 5
        pass


In [52]:
class Network():

    def __init__(self, loss, optimizer, metrics = []):
        self.layers = []
        self.loss = loss
        self.optimizer = optimizer
        self.metrics = metrics

    def add(self, layer):
        self.layers.append(layer)

    def fit(self, X, t, epochs, batch_size=256, print_stats=False):
        X = np.array(X)
        t = np.array(t)
        X = X.reshape(len(X), -1)
        t = t.reshape(len(t), -1)
        if X.shape[0] != t.shape[0]:
            raise ValueError("Array sizes don't match")

        for epoch in range(epochs):
            if print_stats:
                print("Epoch %d" % (epoch+1))
                print("    -> batch size: %d" % batch_size)
            rng_state = np.random.get_state()
            np.random.shuffle(X)
            np.random.set_state(rng_state)
            np.random.shuffle(t)
            pos = 0
            while pos < len(X):
                batch_X = X[pos:pos+batch_size]
                batch_t = t[pos:pos+batch_size]
                self._fit_on_batch(batch_X, batch_t)
                pos += batch_size
            if print_stats:
                _, y = self._forward_pass(X)
                l = self.loss.forward_pass(y, t)
                print("    -> loss: %f" % l)
                for m in self.metrics:
                    print("    -> %s: %f" % (m.__name__, m(y, t)))

    def predict(self, X):
        inps, out = self._forward_pass(X)
        return out

    def _fit_on_batch(self, batch_X, batch_t):
        inps, out = self._forward_pass(batch_X)
        print self.loss, type(out)
        layer_grads = self._backward_pass(
            inps,
            self.loss.backward_pass(out, batch_t)
        )
        print len(inps), len(layer_grads)
#         shapes = lambda x: [a.shape for a in x]
#         print shapes(inps), shapes(layer_grads)
        grad = self._join(layer_grads)
        deltas = self.optimizer.calculate_deltas(grad)
        for l, d in zip(self.layers, self._split(deltas, layer_grads)):
            if not d is None:
                l.update_weights(d)

    def _join(self, grads):
        return np.concatenate([g for g in grads if not g is None])

    def _split(self, grads, layer_grads):
        out = []
        start = 0
        for l in layer_grads:
            if l is None:
                out.append(None)
            else:
                out.append(grads[start:start+len(l)])
                start += len(l)
        return out

    def _forward_pass(self, X):
        # Ćwiczenie 2
        inps = [X]
        output = None
        for i in range(1,len(self.layers)):
            inps.append(self.layers[i].forward_pass(inps[i-1]))
        output = inps.pop()
        return inps, output

    def _backward_pass(self, inps, grad):
        # Ćwiczenie 3
        layer_grads = [grad]
        length = len(inps)
        print len(inps)
        for i in range(1, len(self.layers)):
            lr_ind = length - i
            print lr_ind
            print inps[lr_ind]
            res, _ = self.layers[lr_ind].\
                     backward_pass(inps[lr_ind], layer_grads[0])
            
            layer_grads = [res] + layer_grads
        return layer_grads

    def _debug_grads(self, X, t):
        layer_grads = []
        for l in self.layers:
            g = l.debug_grad(
                lambda: self.loss.forward_pass(self._forward_pass(X)[1], t)
            )
            if not g is None:
                g = np.array(np.array(g).flat)
            layer_grads.append(g)
        return layer_grads

In [60]:
# Ćwiczenie 4
model = Network(MSE(), GD(0.1))
model.add(Dense(28*28, 28*28))
model.add(Sigmoid())
print X_train.shape, y_train.shape
model.fit(X_train, y_train, 10)

(11339, 784) (11339,)
<__main__.MSE instance at 0x1323b9fc8> <type 'numpy.ndarray'>
1
0
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
1 2
[(256, 784)] [(256, 784), (256, 784)]
(256, 784) (785, 784)


ValueError: total size of new array must be unchanged

In [ ]:
# Ćwiczenie 7

In [ ]:
# Ćwiczenie 8

In [ ]:
# Ćwiczenie 9